In [2]:
import os
import sys
import re
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import copy
import pickle
from scipy.optimize import fmin

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sys.path.append('../../statmechlib')
from statmechlib.read_write import read_vasp
from statmechlib.preprocessing import Trajectory, select_nodes, scale_configuration, pair_dist_cutoff
from statmechlib.preprocessing import pair_dist, force_targ, get_stats_EAM, get_stats_EAM_pairdist
from statmechlib.forcefields import sd2_loss, utot_EAM, ftot_EAM, udif_print, u_core
from statmechlib.preprocessing import universal_eos

In [5]:
target_raw = '../data/target_raw'
target_proc = '../data/target_processed'
working = '../data/working'

In [6]:
it = 2 # read in it (independent process)

In [7]:
n = int((1.45-0.85)/0.05 + 0.01)
reos = np.linspace(0.85, 1.45, n+1)*2.74

In [8]:
with open(os.path.join(working, 'stats_clean'+'.pickle'), 'rb') as fi:
    stats_ready = pickle.load(fi)

with open(os.path.join(working, 'target_clean'+'.pickle'), 'rb') as fi:
    targ_ready = pickle.load(fi)   

In [9]:
multi_index = []
multi_index.append([0, 3, 5, 8, 13, 17, 20, 24, 27, 31])
multi_index.append([2, 4, 8, 11, 13, 17, 20, 24, 27, 32])
multi_index.append([1, 4, 6, 9, 13, 17, 20, 24, 27, 31])
multi_index.append([1, 4, 7, 9, 13, 17, 20, 24, 27, 31])
multi_index.append([1, 3, 5, 8, 12, 16, 19, 23, 28, 31])
multi_index.append([0, 4, 6, 8, 12, 17, 20, 25, 28, 32])

In [10]:
stats_opts = []
for index in multi_index:
    index = np.array([True if i in index else False for i in range(len(stats_ready['hyperparams']))])
    print(np.array(stats_ready['hyperparams'])[index])
    stats_opts.append(select_nodes(stats_ready, index))

[2.4  2.65 2.72 2.75 3.   3.45 4.   4.5  5.   5.6 ]
[2.5  2.7  2.75 2.85 3.   3.45 4.   4.5  5.   5.75]
[2.45 2.7  2.73 2.77 3.   3.45 4.   4.5  5.   5.6 ]
[2.45 2.7  2.74 2.77 3.   3.45 4.   4.5  5.   5.6 ]
[2.45  2.65  2.72  2.75  2.9   3.3   3.75  4.356 5.15  5.6  ]
[2.4  2.7  2.73 2.75 2.9  3.45 4.   4.65 5.15 5.75]


In [11]:
#pars_dict['pair'] = [-28.301652628205268, -0.0022071786576090694, 0.027656591839734557, -6.543090393280296, 10.102407371395133, -2.478121982724172, 3.7048200337887582, -3.6346043391919514, 1.7181534576100437, -0.2520201994640249]
#pars_dict['embed'] = [-1.1825596913794016, 2.8944059727957714e-05]
pars_list = []
for index in multi_index:
    pars_dict = {}
    if len(np.array(stats_ready['hyperparams'])[index]) == 10:
        pars_dict['embed'] = [-6.84229349e-01,  1.52487644e-04]
        pars_dict['pair'] = [-6.56958142e+01, 0.0, 2.30194161e+01, 0.0, -3.48639263e+00,  3.45615281e+00, -8.88083580e-01, -1.34315802e+00, 1.24532843e+00, -3.51200974e-01]
    elif len(np.array(stats_ready['hyperparams'])[index]) == 9:
        pars_dict['embed'] = [-6.84229349e-01,  1.52487644e-04]
        pars_dict['pair'] = [-6.56958142e+01, 0.0, 2.30194161e+01, -3.48639263e+00,  3.45615281e+00, -8.88083580e-01, -1.34315802e+00, 1.24532843e+00, -3.51200974e-01]        
    elif len(np.array(stats_ready['hyperparams'])[index]) == 8:
        pars_dict['embed'] = [-6.84229349e-01,  1.52487644e-04]
        pars_dict['pair'] = [-6.56958142e+01, 2.30194161e+01, -3.48639263e+00,  3.45615281e+00, -8.88083580e-01, -1.34315802e+00, 1.24532843e+00, -3.51200974e-01]        
    else:
        raise ValueError('Unsupported number of parameters')
    print(len(np.array(stats_ready['hyperparams'])[index]), len(pars_dict['pair']))
    pars_list.append(pars_dict)

with open(os.path.join(working, "pars_in.pickle"), 'wb') as fo:
    pickle.dump(pars_list, fo)


10 10
10 10
10 10
10 10
10 10
10 10


In [12]:
# read corresponding parameters
with open(os.path.join(working, 'pars_in'+'.pickle'), 'rb') as fi:
    pars_list = pickle.load(fi)
print(pars_list)

[{'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974]}, {'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974]}, {'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974]}, {'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974]}, {'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974]}, {'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974]}]


In [13]:
stats = []
target = []
for key in list(targ_ready.keys()):
    stats.append(stats_opts[it][key])
    target.append(targ_ready[key])

In [14]:
pars_dict = pars_list[it]
pars_in = pars_dict['embed'] + pars_dict['pair']
print(pars_in, len(pars_in))

[-0.684229349, 0.000152487644, -65.6958142, 0.0, 23.0194161, 0.0, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 1.24532843, -0.351200974] 12


In [15]:
target[0]['weight'] = 1.0
target[1]['weight'] = 2.0
target[2]['weight'] = 0.5
target[3]['weight'] = 1.0
target[4]['weight'] = 0.8

In [16]:
len(target[0]['energy']), [t['weight'] for t in target], target[1]['energy']

(1284,
 [1.0, 2.0, 0.5, 1.0, 0.8],
 [-1139.2,
  -906.9778906249999,
  -1127.13044453125,
  -1136.3561554687499,
  -1136.7059554687498,
  0.0,
  0.0])

In [17]:
def optimize_EAM(target, stats, ieam, multi_pars, n_b=3):
    
    optimal_parameters = []
    
    for i, pars in enumerate(multi_pars):
        sd_ini = sd2_loss(list(pars), stats, target, utot_EAM, None, [ieam])
        print(i,' sd2: ini', sd_ini, end='')
        output = fmin(sd2_loss, pars, args=(stats, target, utot_EAM, None, [ieam]), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)
        optimal_parameters.append(tuple([output[1], output[0]]))
        print('sd2: final', output[1])

    n_best = min(n_b, len(optimal_parameters))
    #print(optimal_parameters)
                 
    best_params = sorted(optimal_parameters, key=lambda param: param[0])[:n_best]
    
    print('Best params:', best_params)

    return best_params

In [18]:
multi_pars = []

multi_pars.append(pars_in)

multi_pars.append([-1.48107607e+00, -2.11281591e-06, -7.75748552e+01,  1.23284902e+02,
       -1.05876828e+02,  2.56371829e+01,  7.21885424e-01, -2.01549825e+00,
        9.46930507e-01,  2.72867578e-01, -1.23624777e+00,  4.90853203e-01])

multi_pars.append([ 4.21813176e+00,  1.28438711e-02,  7.18447209e+02,  2.79873504e+01,
       -3.64063120e+01,  1.55668343e+01, -3.59671207e+00,  7.18079901e-02,
       -7.56657066e-02,  2.53180527e-01, -4.96952030e-01, -4.27379804e-02])

multi_pars.append([-1.70284536e+00,  1.77584742e-03, -2.03091714e+02,  6.67172987e+02,
       -1.39072153e+03,  7.95275451e+02, -2.44611978e+01,  5.09001525e+00,
       -3.57713253e-01, -1.04584266e+00,  1.30255105e+00, -4.09472026e-01])

multi_pars.append([ 1.49705729e+00,  8.14531512e-03, -8.15988329e+02,  5.87519886e+02,
       -1.01285873e+03,  5.10026120e+02, -1.11686150e+01,  1.51343754e+00,
        1.15612472e-01, -4.84858749e-01,  3.56870076e-01, -2.09915075e-01])

multi_pars.append([ 1.54864560e+00,  1.06062750e-02, -3.00996729e+02,  3.32437763e+02,
       -3.39714171e+02,  9.93126859e+01, -2.04873166e+01,  1.12932344e+00,
       -7.73962738e-02, -1.97622239e-01,  1.14699068e-01, -1.23093792e-01])

multi_pars.append([ 1.08153442e+00,  2.12340093e-03, -2.83828424e+02,  3.30196594e+02,
       -3.61260411e+02,  1.30278313e+02, -3.12845720e+01,  2.97822976e+00,
       -5.85372576e-02, -8.72679252e-01,  5.95310242e-01, -2.30722232e-01])

multi_pars.append([-1.84530425e+00, -2.18389467e-05, -1.46520729e+03,  1.16790147e+02,
       -1.41528209e+02,  7.82448301e+01,  4.04340173e+00, -2.96638562e+00,
        2.19448488e+00, -5.43928853e-01, -1.31154657e+00,  7.46848321e-01])


In [19]:
for ieam in range(6, len(stats_opts[it]['hyperparams'])):
    print('ieam:', ieam)
    m_pars = multi_pars
    for i in range(6):
        best_pars = optimize_EAM(target, stats, ieam, m_pars, n_b=len(m_pars))
        m_pars = [p[1] for p in best_pars]

ieam: 6
0  sd2: ini 3.9671150898483902sd2: final 0.02317506542253342
1  sd2: ini 5.234494034230205sd2: final 0.028049224065999044
2  sd2: ini 0.06259740668681538sd2: final 0.005185473014370407
3  sd2: ini 0.3439890442928861sd2: final 0.01857813982165249
4  sd2: ini 0.2618545956892867sd2: final 0.005164585631182657
5  sd2: ini 2.1904453042896574sd2: final 0.005745104783258027
6  sd2: ini 4.189872556697367sd2: final 0.008168609770510805
7  sd2: ini 6.705263865214785sd2: final 0.02967775990412804
Best params: [(0.005164585631182657, array([ 2.31646724e+00,  1.36982545e-02, -4.03597003e+02,  2.24509647e+02,
       -2.78263860e+02,  8.44339473e+01, -6.31012211e+00,  1.15436868e-02,
       -3.66159467e-01,  1.69961045e-01, -8.84445288e-02, -1.32995741e-01])), (0.005185473014370407, array([ 3.09282254e+00,  1.49670755e-02, -4.20895478e+02,  1.75466522e+02,
       -1.94398078e+02,  4.62458200e+01, -4.79252754e+00, -5.00636982e-01,
       -4.50799402e-01,  4.92247093e-01, -4.04425267e-01, -7.14

0  sd2: ini 0.005005351979446691sd2: final 0.005005351979446168
1  sd2: ini 0.00500535237651526sd2: final 0.005005352175720949
2  sd2: ini 0.0050054278741481105sd2: final 0.005005360993382526
3  sd2: ini 0.005005634508171659sd2: final 0.005005458359607944
4  sd2: ini 0.0050073201493656015sd2: final 0.005005741031530928
5  sd2: ini 0.00502401058780556sd2: final 0.005005460435499647
6  sd2: ini 0.005029481473794042sd2: final 0.005015334911908957
7  sd2: ini 0.005538319516623085sd2: final 0.005092442915910012
Best params: [(0.005005351979446168, array([ 2.73321931e+00,  1.39273618e-02, -5.23898709e+02,  2.77581081e+02,
       -3.38495570e+02,  9.96028161e+01, -5.81785973e+00, -1.39078673e-01,
       -4.00865767e-01,  3.32838985e-01, -2.54503288e-01, -1.00311563e-01])), (0.005005352175720949, array([ 2.73304350e+00,  1.39275856e-02, -5.23810673e+02,  2.77481231e+02,
       -3.38353396e+02,  9.95511029e+01, -5.81728794e+00, -1.39220687e-01,
       -4.00699667e-01,  3.32629642e-01, -2.543669

0  sd2: ini 0.01303472566695436sd2: final 0.013025010453169006
1  sd2: ini 0.013158511834013491sd2: final 0.01308506903164834
2  sd2: ini 0.013161645965981975sd2: final 0.013120702752029225
3  sd2: ini 0.013208511684005624sd2: final 0.013029816791614475
4  sd2: ini 0.013460338976193639sd2: final 0.013099552623482957
5  sd2: ini 0.014489762827767323sd2: final 0.013024998922211884
6  sd2: ini 0.01672768033912245sd2: final 0.013025008897041886
7  sd2: ini 0.02193913049998275sd2: final 0.01535983491621373
Best params: [(0.013024998922211884, array([ 2.17750532e+00,  2.68194214e-03, -7.48185741e+02,  6.20007624e+02,
       -8.97803129e+02,  3.49145549e+02, -1.53968056e+01,  2.69817592e+00,
       -6.29408575e-01, -3.63007397e-01,  4.79262122e-01, -3.57727318e-01])), (0.013025008897041886, array([ 2.17763831e+00,  2.68216259e-03, -7.48545465e+02,  6.20028106e+02,
       -8.97727214e+02,  3.49068882e+02, -1.53930233e+01,  2.69702841e+00,
       -6.27687937e-01, -3.64863708e-01,  4.80224198e-0

0  sd2: ini 3.688858246402586sd2: final 0.013764183162972581
1  sd2: ini 3.103153017743889sd2: final 0.013117955543000936
2  sd2: ini 7.229683206370137sd2: final 0.018401560759775937
3  sd2: ini 4.833711361996873sd2: final 0.03754626439681316
4  sd2: ini 7.205783917980599sd2: final 0.03773586726742029
5  sd2: ini 7.219126126872632sd2: final 0.01458687277095825
6  sd2: ini 6.9226170172314125sd2: final 0.03471919290242599
7  sd2: ini 5.8010822758319005sd2: final 0.013452837182540839
Best params: [(0.013117955543000936, array([-2.32531763e+00, -2.30493495e-06, -6.00131842e+02,  3.92348604e+02,
       -5.13642130e+02,  1.82754913e+02, -3.34031090e+00,  1.62278584e-01,
        8.35527644e-01, -1.13926995e+00,  2.43596349e-01,  1.82370719e-01])), (0.013452837182540839, array([-2.28137547e+00, -1.27155155e-05, -1.47480701e+02,  1.88618688e+02,
       -2.95376674e+02,  1.36930468e+02, -5.74304553e+00,  1.06781611e+00,
        3.37503920e-01, -8.39405064e-01,  1.55247575e-01,  1.85439981e-01]))

0  sd2: ini 0.011995149351694656sd2: final 0.011993727287430485
1  sd2: ini 0.011995751941032577sd2: final 0.011994969931312403
2  sd2: ini 0.012002504603962107sd2: final 0.011998361430577552
3  sd2: ini 0.012032512285025308sd2: final 0.012005992198904674
4  sd2: ini 0.012058827171062924sd2: final 0.011998924017078779
5  sd2: ini 0.012064511740672064sd2: final 0.012024210726687334
6  sd2: ini 0.012104175361930111sd2: final 0.012068396124456789
7  sd2: ini 0.016612115502896346sd2: final 0.012368639991658173
Best params: [(0.011993727287430485, array([-3.11077769e+00, -2.56029762e-04, -3.29892271e+02,  4.16677419e+02,
       -5.79220751e+02,  2.18138378e+02, -1.80655049e+00, -8.53912206e-02,
        1.00508952e+00, -9.74422186e-01, -2.89865823e-02,  3.53865794e-01])), (0.011994969931312403, array([-3.06847025e+00, -2.47622386e-04, -3.37623938e+02,  4.20241720e+02,
       -5.84247741e+02,  2.20003755e+02, -1.95305333e+00, -4.48889950e-02,
        9.91939913e-01, -9.77847071e-01, -1.519465

../../statmechlib/statmechlib/forcefields/eam.py:226: RuntimeWarning: overflow encountered in exp
  eee = np.exp(-uuu)


sd2: final 12.268686118726542
3  sd2: ini 7.007561268782704sd2: final 0.27571070295749606
4  sd2: ini 7.229780643756137sd2: final 0.032279700097404364
5  sd2: ini 11.542857511745435sd2: final 11.542857511745435
6  sd2: ini 7.2291649111373175sd2: final 0.015511463818139682
7  sd2: ini 0.11890416055165061sd2: final 0.008144465994776197
Best params: [(0.008144465994776197, array([-1.43157386e+00, -3.48918395e-06, -3.81795757e+02,  3.76959890e+02,
       -4.92775498e+02,  1.68968794e+02,  7.83350640e-01, -1.76038225e+00,
        1.44862322e+00, -3.20553311e-01, -9.59047517e-01,  5.23912873e-01])), (0.008563229662321844, array([-1.47942839e+00,  3.52632770e-07,  5.14451305e+01,  1.97450212e+02,
       -2.80212943e+02,  1.06842804e+02,  1.38373756e+00, -1.97190514e+00,
        1.48160584e+00, -3.25276680e-01, -1.00076115e+00,  5.46714843e-01])), (0.010199680843967, array([-1.22757797e+00,  2.74862966e-05, -1.75583680e+02,  1.59097307e-02,
        1.84438586e+01, -2.01988127e-02, -2.49310327e

0  sd2: ini 0.008138728045483925sd2: final 0.008138727136294883
1  sd2: ini 0.008142223492433908sd2: final 0.008139266360165577
2  sd2: ini 0.008147775087339073sd2: final 0.008143789821685376
3  sd2: ini 0.008173472965188338sd2: final 0.008142841621287344
4  sd2: ini 0.008180406275964374sd2: final 0.008150289760179438
5  sd2: ini 0.008319747279655856sd2: final 0.008235795534950817
6  sd2: ini 11.542857511745435sd2: final 11.542857511745435
7  sd2: ini 12.268686118726542sd2: final 12.268686118726542
Best params: [(0.008138727136294883, array([-1.40692793e+00, -6.36236637e-07, -3.56002663e+02,  3.58438114e+02,
       -4.70296439e+02,  1.62530898e+02,  6.05475392e-01, -1.69412620e+00,
        1.41147005e+00, -3.24884565e-01, -9.32817233e-01,  5.10338937e-01])), (0.008139266360165577, array([-1.41924228e+00, -1.66080045e-06, -3.59219740e+02,  3.61410065e+02,
       -4.73706309e+02,  1.63387544e+02,  6.76832505e-01, -1.72076655e+00,
        1.42274981e+00, -3.20162047e-01, -9.46650365e-01, 

In [20]:
optimal_parameters = [(4.407570269501649e-05, [-1.21658318e+00,  3.24863580e-05, -4.73661687e+01,  2.29965951e+01,
       -3.42248343e+00,  3.73956980e+00, -1.39292737e+00, -8.33458565e-01,
        9.29769431e-01, -1.83568989e-01]), (6.17447812420396e-05, [-1.30551487e+00,  1.41920657e-05, -4.61160967e+01,  2.29352328e+01,
       -3.42360958e+00,  3.82313395e+00, -1.54715703e+00, -6.97925842e-01,
        8.68730239e-01, -1.54592379e-01]), (9.723561709685708e-05, [-1.34624502e+00,  7.49457487e-06, -4.58871674e+01,  2.24064873e+01,
       -3.46516902e+00,  4.02263760e+00, -1.89661680e+00, -4.31617423e-01,
        8.06406324e-01, -1.37562101e-01]), (0.00014123622250142564, [-1.34136558e+00,  1.01769934e-05, -4.93998032e+01,  2.21279115e+01,
       -3.50791272e+00,  4.16822104e+00, -2.13414308e+00, -2.66237999e-01,
        7.83819577e-01, -1.36703203e-01])]

In [21]:
optimal_parameters = [ieam9]

NameError: name 'ieam9' is not defined

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['relax']['xyz']])[:-2]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, target, utot_EAM, [9])
    plt.title('Selected configurations at 0K')
    if i == 0:
        print('Targ:', targ_out[-4][:-2])
        print('Model:', opti_out[-4][:-2])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[-4][:-2]))/nat)
        plt.plot(np.array(targ_out[-4][:-2])/nat, 'o', label='target'+str(i), markersize=10)
        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    print('Model:', opti_out[-4][:-2])

    #if i != 10 and i != 12:
    #    continue
    print(i, params_uopt[0])

        
    #print('Opti:', [o for o in opti_out[-1][:-1]])
    plt.plot(np.array([o for o in opti_out[-4][:-2]])/nat, 'x--', label='optimized '+str(i))
#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]')
#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.7533)
#plt.ylim(-8.9, -8.35)
#plt.xlim(0, 4)

plt.legend()

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['eos_bcc']['xyz']])[:-1]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, target, utot_EAM, [9])
    plt.title('Selected configurations at 0K')
    if i == 0:
        print('Targ:', targ_out[-2][:])
        print('Model:', opti_out[-2][:])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[-2][:-1]))/nat)
        plt.plot(np.array(targ_out[-2][:-1])/nat, 'o', label='target'+str(i), markersize=10)
        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    #if i != 10 and i != 12:
    #    continue
    print(i, params_uopt[0])

        
    #print('Opti:', [o for o in opti_out[-1][:-1]])
    plt.plot( np.array([o for o in opti_out[-2][:-1]])/nat, 'x--', label='optimized '+str(i))
#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

#plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]')
#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.7533)
#plt.ylim(-8.9, -8.35)
#plt.xlim(0, 4)

plt.legend()

In [ ]:
len(targ_ready['md']['xyz'])

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['md']['xyz']])[:]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, target, utot_EAM,[9])
    #plt.title('Selected configurations at 0K')
    if i == 0:
        #print('Targ:', targ_out[0][:])
        #print('Model:', opti_out[0][:])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[0][:]))/nat)
        #plt.plot(np.array(targ_out[0][:])/nat, 'o', label='target'+str(i), markersize=10)
        plt.plot(np.array(targ_out[0][:]), 'o', label='target'+str(i), markersize=5)

        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    #if i != 1:# and i != 12:
    #    continue
    print(i, params_uopt[0])

    #print('Opti:', [o for o in opti_out[-1][:-1]])
    #plt.plot(np.array([o for o in opti_out[0][:]])/nat, 'x--', label='optimized '+str(i))
    plt.plot(np.array([o for o in opti_out[0][:]]), '--', label='optimized '+str(i), lw=1.0)

#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

#plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]', fontsize=30)
plt.xlabel('configuation #', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.533)
#plt.ylim(-8.9, -8.35)

#plt.ylim(-1075,-1035)
#plt.ylim(-1000,-900)
#plt.ylim(-950,-900)
#plt.ylim(-1136,-1133)
plt.ylim(-1180,-800)
plt.xlim(0,820)
#plt.xlim(847,990)

#plt.ylim(-1180,-800)

plt.legend(fontsize=30)
plt.savefig(plt.savefig(os.path.join(working, 'bcc_300.png')))

In [ ]:
for i, params_uopt in enumerate(optimal_parameters):
    if params_uopt[0] > 1.1e-3:
        continue
    print(i, params_uopt[0], '\n', params_uopt[1])

In [ ]:
stats_opts[it]['hyperparams']

In [ ]:
pars_out = {}
pars_out['pair_nodes'] = stats_opts[it]['hyperparams']
pars_out['mb_node'] = 6
pars_out['params'] = [optimal_parameters[1]]

In [ ]:
optimal_list = []

#with open(os.path.join(working, 'pars_out.pickle'), 'rb') as fi:
#    optimal_list = pickle.load(fi)

#if type(optimal_list) != list:
#    optimal_list = [optimal_list]

optimal_list.append(pars_out)

with open(os.path.join(working, 'pars_out.pickle'), 'wb') as fo:
    pickle.dump(optimal_list, fo)

In [ ]:
with open(os.path.join(working, 'pars_out.pickle'), 'rb') as fi:
    optimal_list = pickle.load(fi)

In [ ]:
optimal_list

In [ ]:
# Total pair potential
pos = targ_ready['relax']['xyz'][0]
bx = targ_ready['relax']['box'][0]
rr, rx = pair_dist_cutoff(pos, bx, 5.5)

In [ ]:
# embedding function
F = lambda d, a: a[0]*d**0.5 + a[1]*d + a[2]*d**2

V = lambda r, aa, cc: sum([a*(rc - r)**3 for a, rc in zip(aa, cc) if r < rc])

# Cohesive potential (cubic splines - same form as V)
phi = lambda r, aa, cc: sum([a*(rc - r)**3 for a, rc in zip(aa, cc) if r < rc])

rho = lambda r, aa, kk: sum([a*(rk - r)**3 for a, rk in zip(aa, kk) if r < rk and r > 0.01])

In [ ]:
optimal_list[-1]['params'][0][1]

In [ ]:
#optimal_parameters.append(tuple([output[1:], params_uopt]))
pair_y = optimal_list[-1]['params'][0][1][2:]
par = optimal_list[-1]['params'][0][1][:2]
many_y = [par[0], 0.0, par[1]]
V_x_r = optimal_list[-1]['pair_nodes'] #stats_opt['hyperparams']
rho_x_r = [V_x_r[optimal_list[-1]['mb_node']]] #stats_opt['hyperparams'][4:5]
rho_x_a = [1.0]

In [ ]:
pair_y, many_y, V_x_r, rho_x_r

In [ ]:
ene = []
for a in np.linspace(0.5, 2.0, 101):
    z = a*rr
    u_pair = 0.5*np.sum([V(x, pair_y, V_x_r) for x in z.flatten() if x > 0.1])

    u_many = 0.0
    for i in range(z.shape[0]):
        d = np.sum([phi(x, rho_x_a, rho_x_r) for x in z[i] if x > 0.1])
        u_many += F(d, many_y)

    ene.append([a, u_pair, u_many, u_pair+u_many])

ene = np.array(ene)

In [ ]:
#plt.plot(ene[:,0], ene[:,1]/54, label='pair')
#plt.plot(ene[:,0], ene[:,2]/54, label='manybody')
plt.title('Equation of state (T=0K)')
plt.plot(ene[:,0]*2.74, [universal_eos(x, 'W') for x in ene[:,0]], label='universal EOS')
plt.plot(ene[:,0]*2.74, ene[:,3]/128, label='u-total')
plt.hlines(0.0,0.5,6.0,lw=1)
plt.legend()
plt.xlim(2.0, 5.75)
plt.xlim(2.6, 2.85)
plt.ylim(-10,10)
plt.ylim(-9.,-8.5)

plt.xlabel('x')
plt.ylabel(r'U')

In [ ]:
#optimal_parameters.append(tuple([output[1:], params_uopt]))
pair_y = optimal_list[-1]['params'][0][1][2:]
par = optimal_list[-1]['params'][0][1][:2]
many_y = [par[0], 0.0, par[1]]
V_x_r = optimal_list[-1]['pair_nodes'] #stats_opt['hyperparams']
rho_x_r = [V_x_r[optimal_list[-1]['mb_node']]] #stats_opt['hyperparams'][4:5]
rho_x_a = [1.0]

In [ ]:
V_x_r

In [ ]:
pair_a = optimal_list[-1]['params'][0][1][2:]#pars_in1[2:]
pair_r = optimal_list[-1]['pair_nodes']

rho_a = [1.0]
rho_r = [pair_r[optimal_list[-1]['mb_node']]]

par = optimal_list[-1]['params'][0][1][:2]
many_a = [par[0], 0.0, par[1]]

In [ ]:
# Ranges of densities and separations

nr = 5000
rcut = pair_r[-1]#stats_opt['hyperparams'][-1]
dr = rcut/nr

rhomax = 12.0*phi(1.5, rho_a, rho_r)
print('rhomax:', rhomax)
rhomax = 1000.0
nrho = 5000
drho = rhomax/nrho

ri = 1.0
ro = 2.0

# W properties
na = 74
ma = 183.84
lc = 3.1653
alat = 'bcc'

# System composition
n_el = 1
el_name = ['W']

In [ ]:
def u_trans(r, unucl, ueq, ri=1.0, ro=2.0):
    """Transition between the core and equillibrium potentials"""
    if r < ri:
        return unucl
    elif r < ro:
        x = (ro + ri - 2*r)/(ro - ri)
        eta = 3/16*x**5 - 5/8*x**3 + 15/16*x + 1/2
        return ueq + eta*(unucl - ueq)
    else:
        return ueq

In [ ]:
# prepare lists of values
dens = np.linspace(0., rhomax-drho, nrho, endpoint=True)
r = np.linspace(0., rcut-dr, nr, endpoint=True)
r2 = np.linspace(0., rcut, 201)

fembd = [F(d, many_a) for d in dens] # W
edens = np.array([phi(x, rho_a, rho_r) for x in r])

# 1. Cubic spline for r > r_o
f_outer = [V(x, pair_a, pair_r) for i, x in enumerate(r)]
# 2. Repulsive core for r < r_i (precalculate up to r_o)
f_inner = [0.0] + [u_core(x) for x in r[1:]]
# 3. Transition region for r_i < r < r_o
fpair = [x*u_trans(x, f_inner[i], f_outer[i]) for i, x in enumerate(r)]
#fpair = f_outer

In [ ]:
# LAMMPS format
pot_path = '../sim/potentials'

with open(os.path.join(pot_path, 'W_lv_3.eam.fs'), 'w') as fo:
    
    # Comment section
    fo.write('Comment 1\n')
    fo.write('Comment 2\n')
    fo.write('Comment 3\n')
    
    # Number of elements
    fo.write(f"{n_el:5d} ")
    for i in range(n_el):
        fo.write(f"{el_name[i]:2}")
    fo.write("\n")
    
    # number of grid points, step size, and cutoff
    fo.write(f"{nrho:5d}{drho:24.16e}{nr:5d}{dr:24.16e}{rcut:24.16e}\n")
    
    # atomic number, mass, lattice size, lattice type
    fo.write(f"{na:5d} {ma:14.4f} {lc:14.4f} {alat:10}\n")
    
    # Embeding function
    for i in range(nrho//4):
        fo.write("{0:20.12e}{1:20.12e}{2:20.12e}{3:20.12e}\n".format(*fembd[i*4:i*4+4]))
    
    # Electronic density
    for i in range(nr//4):
        fo.write("{0:20.12e}{1:20.12e}{2:20.12e}{3:20.12e}\n".format(*edens[i*4:i*4+4]))
        
    # Pair potential
    for i in range(nr//4):
        fo.write("{0:20.12e}{1:20.12e}{2:20.12e}{3:20.12e}\n".format(*fpair[i*4:i*4+4]))